In [1]:
import os
import tensorflow as tf
from dataclasses import dataclass
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [2]:
%pwd

'/Users/arpansethi/Documents/ai-projects/kidney-disease/Kidney-Disease-Classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/arpansethi/Documents/ai-projects/kidney-disease/Kidney-Disease-Classification'

In [5]:
dagshub.init(repo_owner='arpansethi30', repo_name='Kidney-Disease-Classification', mlflow=True)

[2024-10-05 02:59:36,704: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as arpansethi30

[2024-10-05 02:59:36,710: INFO: helpers: Accessing as arpansethi30]
[2024-10-05 02:59:37,214: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/arpansethi30/Kidney-Disease-Classification "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "arpansethi30/Kidney-Disease-Classification"

[2024-10-05 02:59:37,220: INFO: helpers: Initialized MLflow to track repo "arpansethi30/Kidney-Disease-Classification"]


Repository arpansethi30/Kidney-Disease-Classification initialized!

[2024-10-05 02:59:37,222: INFO: helpers: Repository arpansethi30/Kidney-Disease-Classification initialized!]


In [6]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Kidney_Disease",
            mlflow_uri="https://dagshub.com/arpansethi30/Kidney-Disease-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [8]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.tensorflow.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.tensorflow.log_model(self.model, "model")


In [9]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-05 02:59:37,249: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-05 02:59:37,252: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-05 02:59:37,253: INFO: common: created directory at: artifacts]


2024-10-05 02:59:37.272790: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-10-05 02:59:37.272828: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-10-05 02:59:37.272833: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-10-05 02:59:37.272864: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-05 02:59:37.272877: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 2207 images belonging to 2 classes.
  1/138 [..............................] - ETA: 36s - loss: 7.6665e-06 - accuracy: 1.0000

2024-10-05 02:59:37.759153: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


138/138 [==============================] - 7s 50ms/step - loss: 1.1047 - accuracy: 0.8446
[2024-10-05 02:59:44,780: INFO: common: json file saved at: scores.json]


2024/10/05 02:59:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/hv/392wjqgd1j34prxvqfc8zwwc0000gn/T/tmpydiz305d/model/data/model/assets
[2024-10-05 02:59:47,041: INFO: builder_impl: Assets written to: /var/folders/hv/392wjqgd1j34prxvqfc8zwwc0000gn/T/tmpydiz305d/model/data/model/assets]


2024/10/05 02:59:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/05 03:00:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 6
Created version '6' of model 'VGG16Model'.
2024/10/05 03:00:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run brawny-mule-837 at: https://dagshub.com/arpansethi30/Kidney-Disease-Classification.mlflow/#/experiments/0/runs/030095d4537a4cbcb446a4ec2b7aa6fa.
2024/10/05 03:00:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/arpansethi30/Kidney-Disease-Classification.mlflow/#/experiments/0.
